# Viterbi version of target sequence labelling
# Intro
The viterbi algorithm for labelling sequences uses the Hidden-Markov-Model and computes the
most probable label sequence as the combination of transition and emission probabilities for
the given word sequence. It is most often used in POS-tagging but nicely fits our task of
labelling emotion target sequences. As it uses not only the observed tokens but also the last
(number of) tag, it takes the probability of tags following each other into account. As our tags
are rigid in their sequence (Inside only follows Beginning, Beginning only follows Outside), using
transition probabilities will improve the output labels by hopefully avoiding illegal tag transitions.
# Method
The used viterbi algorithm is implemented from scratch and works on the non-preprocessed tokens.
To use it, the data has to be prepared first.

In [2]:
import pandas

## Data preparation
As seen later, the algorithm is trained by calculating emission and transition probabilities,
which are taken from the tokenized training data. First, the file is loaded and all unnecessary
parts are dropped.

In [4]:
with open('emotion-rl-corpora.jsonl', encoding='utf-8') as f:
    data_source = pandas.read_json(f, lines=True)
data_source = data_source.loc[
    (data_source['dataset']=='reman')|(data_source['dataset']=='electoral_tweets')|(data_source['dataset']=='gne')
]

data_source = data_source.drop(columns=[
    'meta', 'steps', 'tags', 'split', 'annotation-offsets', 'extra'
])

data_source = data_source.join(data_source['annotations'].apply(pandas.Series))

data_source = data_source.drop(columns=['annotations', 'cause', 'cue-joy', 'cue-sadness', 'cue-anger', 'cue-other', 'cue-disgust',
    'cue-trust', 'cue-anticipation', 'cue-surprise', 'cue-fear', 'cue', 'experiencer'])

Index(['dataset', 'id', 'emotions', 'text', 'tokens', 'annotations', 'meta',
       'steps', 'tags', 'split', 'annotation-offsets', 'extra'],
      dtype='object')
Index(['dataset', 'id', 'emotions', 'tokens', 'experiencer', 'target',
       'cause'],
      dtype='object')


After removing every annotation but target and all additional information other than the corpus and text id,
the raw text itself and token and label lists, the algorithm can be trained.
## Training
### HMM
The viterbi algorithm is based on a Hidden-Markov model, as the model computes the probability of a token-tag sequence
and viterbi is a way to return the tag sequence with the highest probability. Why viterbi is necessary will be explained
after implementing a HMM.

In [ ]:
class HiddenMarkovModel:
    transitions = dict()
    emissions = dict()
    prior = dict()
    observations = {'transitions': transitions, 'emissions': emissions}

    def __init__(self):
        self.transitions = {'O': {'O': 0, 'B': 0, 'I': 0}, 'B': {'O': 0, 'B': 0, 'I': 0}, 'I': {'O': 0, 'B': 0, 'I': 0}}
        self.prior = {'O': 0, 'B': 0, 'I': 0}

    def train_probabilities(self, train_set: pandas.DataFrame):
        trans = {'O': {'O': 0, 'B': 0, 'I': 0}, 'B': {'O': 0, 'B': 0, 'I': 0}, 'I': {'O': 0, 'B': 0, 'I': 0}}
        emi = dict()
        prior = {'O': 0, 'B': 0, 'I': 0}
        for text in train_set:
            # The current sequence consists of (word, tag) pairs
            sequence = list(zip(train_set[text]['tokens'],train_set[text]['target']))

            for i in range(len(sequence)):
                # First tag: Prior probabilities
                if i == 0:
                    prior[sequence[i][1]] += 1
                else:
                    trans[sequence[i][1]][sequence[i-1][1]] += 1
                    tok = sequence[i][0]
                    tag = sequence[i][1]
                    if tok in emi:
                        emi[tag][tok] += 1
                    else:
                        emi[tag] = {'O': 0, 'B': 0, 'I': 0}
                        emi[tag][tok] += 1
            for tag in {'O', 'B', 'I'}:
                freq = sum(trans[tag].values())
                for prev_tag in trans[tag]:
                    trans[tag][prev_tag] /= freq
                prior[tag] /= sum(prior.values())

            for tok in emi:
                freq = sum(emi[tok].values())
                for tag in {'O', 'B', 'I'}:
                    emi[type][tag] /= freq

            emi['$OOV'] = {tag: sum(trans[tag].values())/sum([sum(trans[tag2].values()) for tag2 in {'O', 'B', 'I'}]) for tag in {'O', 'B', 'I'}}
        self.transitions = trans
        self.emissions = emi
        self.prior = prior

### Viterbi
The trained HMM includes the emission probabilities as token-tag-pair-frequencies relative to token-frequency and
transition probabilities as tag bigram frequencies relative to the frequency of the second tag. As every token in the
sequence has three tag possibilities which are also influenced by the previous tag, there are 3<sup>n</sup> tag sequences
for a sequence with n tokens, all with differing probabilities. Viterbi keeps track of the maximum probability of a current
token and the maximum probability of different histories to compute the best sequence from all HMM-sequences.

In [ ]:
def viterbi(sequence: list[str], hmm: HiddenMarkovModel):
    max_labels = list()
    for i, token in enumerate(sequence):
        if token in hmm.emissions:
            e = hmm.emissions[token]
        if i == 0:
            t = hmm.prior
    # TODO: Viterbi maximum ausrechnen

    return max_labels